---
title: "VRPCD with information exchange"
author: "FA"
date: 2021-01-07
tags: ["VRPCD", "realtime", "information exchange"]
categories: "research"
---

In [77]:
#*****************************************
# Necessary Package
#*****************************************
using Random
using JuMP
using LinearAlgebra
using GLPK

In [78]:
#*****************************************
# Network Property
#*****************************************
# P: set of pickup nodes
P = [2,3]

# D: set of delivery nodes
D = [6,7]

# O: cross-dock (assume single cross dock in the network)
O_pick = [4]
O_del = [5]

# O1: dummy cross-dock
O_dum_pick = [1]
O_dum_del = [8]

# pickup nodes
N_pickup = vcat(O_dum_pick, P, O_pick)

# delivery nodes
N_delivery = vcat(O_del, D, O_dum_del)

# all nodes
N = vcat(O_dum_pick, P, O_pick, O_del, D, O_dum_del)

# M: set of available vehicles
M = ["v1", "v2", "v3", "v4"]

# may be we need to position each entity in a graph to find distance/ time automatically
print(P,D,O_pick, O_del, N_pickup, N_delivery, N, M)


[2, 3][6, 7][4][5][1, 2, 3, 4][5, 6, 7, 8][1, 2, 3, 4, 5, 6, 7, 8]["v1", "v2", "v3", "v4"]

In [79]:
#****************************************
# parameters
#*****************************************
# p = number of pickup nodes
p = length(P)

# d = number of delivery nodes
d = length(D)

# o = number of cross dock
o_pick = length(O_pick)
o_del = length(O_del)


# p_cd: number of nodes in pickup process (dummy CD + P + CD)
p_cd = length(N_pickup)

# d_cd: number of nodes in delivery process (CD + P + dummy CD)
d_cd = length(N_delivery)

# n: total nodes in the network
n = length(N)

# number of availble vehicles
m = length(M)

# p_i: quantity of products to be collected at pickup node i; [include product type in future]
p_i = [0, 30, 50, 0, 0, 0, 0, 0]

# d_i: quantity of products to be delivered at delivery node i; [include product type in future]
d_i = [0, 0, 0, 0, 0, 30, 50, 0]

# capacity of vehicle (homogenous)
Q = 50

# Random.seed!(314)
# c_ij = the travel time (minutes)/cost between node i and node j; (assuming the cost is the same as the time req.)
t_ij= 
    [0 20 30 0 0 0 0 0;
    20 0 50 40 0 0 0 0;
    30 50 0 40 0 0 0 0;
    0 40 40 0 0 0 0 0;
    0 0 0 0 0 70 80 0;
    0 0 0 0 20 0 70 60;
    0 0 0 0 80 70 0 60;
    0 0 0 0 0 60 60 0]

# Cross-dock activity time
T_o = 15

# time horizon (16 hr a day)
T_max = 16*60

960

In [80]:
# ****************************************
# Create a JuMP model
# ****************************************
cd_modl = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [81]:
# ****************************************
# Decision variables
#*****************************************
# x_ijk = 1 if vehicle k travels from node i to node j; 0 otherwise;
@variable(cd_modl, x_ijk[i=1:n, j=1:n, k=1:m], Bin)

# y_i = total quantity of collected/delivered products after leaving node i
@variable(cd_modl, y_i[i = 1:n], Int)

# t_pickup = latest arrival time a cross dock
@variable(cd_modl, t_pick[pick = 1:o_pick])

# AT_k = arrival time of vehicle k at the cross-dock
@variable(cd_modl, AT_k[k =1:m])

4-element Array{VariableRef,1}:
 AT_k[1]
 AT_k[2]
 AT_k[3]
 AT_k[4]

In [82]:
#*****************************************
# Constraints
#*****************************************

# Pick-up Process
#----------------


# [Pick-up Process] 01: only one vehicle can arrive at a pickup node
@constraint(cd_modl, p_node_entry[j= P],
                    sum(x_ijk[i,j,k] for i=vcat(O_dum_pick, P) for k=1:m if i != j) == 1)

#[Pick-up Process] 02: Only one vehicle depart from the pickup node
@constraint(cd_modl, p_node_exit[i= P],
                    sum(x_ijk[i,j,k] for j=vcat(P,O_pick) for k=1:m if i != j) == 1)

# [Pick-up Process] 03: Consecutive movement of vehicles
@constraint(cd_modl, cons_move_pick[l = P, k=1:m],
    sum(x_ijk[i,l,k] for i=vcat(O_dum_pick, P) if i != l) 
    -sum(x_ijk[l,j,k] for j=vcat(P,O_pick) if j != l) 
    == 0)

# [Pick-up Process] 04: vehicle leave at dummy cross dock
@constraint(cd_modl, p_dum_leave[i= O_dum_pick, k=1:m],
    sum(x_ijk[i,j,k] for j = P) <= 1)


# [Pick-up Process] 05: vehicle that leave at dummy cross dock visit cross dock immdiately after last pickup
@constraint(cd_modl, p_cd_entry[j = O_pick, k=1:m],
    sum(x_ijk[i,j,k] for i= P) <= 1)

# [Pick-up Process] 06: 
@constraint(cd_modl, p_pick_qnt[i = P, j = P,  k=1:m; i!=j],
    y_i[j] 
    - y_i[i] 
    - p_i[j] 
    + Q 
    - (Q*(x_ijk[i,j,k] + x_ijk[j,i,k])) 
    + ((p_i[j]+p_i[i])*x_ijk[j,i,k])
    >= 0)

# [Pick-up Process] 07: 
@constraint(cd_modl, p_pick_qnt2[i = O_dum_pick, j = P, k=1:m],
    y_i[j]
    - x_ijk[i,j,k]*p_i[j]
    - (1-x_ijk[i,j,k])*Q
    <= 0)

# [Pick-up Process] 08a: 
@constraint(cd_modl, pick_cap1[i = P],
    y_i[i] >= p_i[i] )

# [Pick-up Process] 08b: 
@constraint(cd_modl, pick_cap2[i = P],
    y_i[i] <= Q)

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, [2, 3]
And data, a 2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 pick_cap2[2] : y_i[2] <= 50.0
 pick_cap2[3] : y_i[3] <= 50.0

In [83]:
#*****************************************
# Constraints
#*****************************************

# Delivery Process
#----------------

# [Delivery Process] 01: only one vehicle can arrive at a pickup node
@constraint(cd_modl, d_node_entry[j= D],
                    sum(x_ijk[i,j,k] for i=vcat(O_del, D) for k=1:m if i != j) == 1)

# [Delivery Process] 02: Only one vehicle depart from the pickup node
@constraint(cd_modl, d_node_exit[i= D],
                    sum(x_ijk[i,j,k] for j=vcat(D,O_dum_del) for k=1:m if i != j) == 1)

# [Delivery Process] 03: Consecutive movement of vehicles
@constraint(cd_modl, cons_move_del[l = D, k=1:m],
    sum(x_ijk[i,l,k] for i=vcat(O_del, D) if i != l) 
    -sum(x_ijk[l,j,k] for j=vcat(D,O_dum_del) if j != l) 
    == 0)

# [Delivery Process] 04: vehicle enter at dummy cross dock after last delivery
@constraint(cd_modl, d_dum_enter[k=1:m],
    sum(x_ijk[i,j,k] for i= D for j = O_dum_del) <= 1)

# [Delivery Process] 05: vehicle leave cross dock
@constraint(cd_modl, d_cd_leave[k=1:m],
    sum(x_ijk[i,j,k] for i= O_del for j = D) <= 1)

# [Delivery Process] 06: 
@constraint(cd_modl, d_del_qnt[k=1:m, i = D, j = D; i!=j],
    y_i[j] 
    - y_i[i] 
    - d_i[j] 
    + Q 
    - (Q*(x_ijk[i,j,k] + x_ijk[j,i,k])) 
    + ((d_i[j]+d_i[i])*x_ijk[j,i,k])
    >= 0)

# [Delivery Process] 07: 
@constraint(cd_modl, d_del_qnt2[i = O_del, j = D, k=1:m],
    y_i[j]
    - x_ijk[i,j,k]*d_i[j]
    - (1-x_ijk[i,j,k])*Q
    <= 0)

# [Delivery Process] 08a: 
@constraint(cd_modl, del_cap1[i = D],
    y_i[i] >= d_i[i] )

# [Delivery Process] 08b: 
@constraint(cd_modl, del_cap2[i = D],
    y_i[i] <= Q)

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, [6, 7]
And data, a 2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 del_cap2[6] : y_i[6] <= 50.0
 del_cap2[7] : y_i[7] <= 50.0

In [84]:
#*****************************************
# Constraints
#*****************************************

# @cross dock:: Connection Between Pickup and Delivery Process (using time)
#------------------------------------------------------------

# [cross-dock] 01: track time for product collection in the pickup process
@constraint(cd_modl, cd_prod_col[k = 1: m],
    AT_k[k]
    -sum(t_ij[i,j] * x_ijk[i,j,k] for i=vcat(O_dum_pick, P) for j=vcat(P, O_pick) if i != j) >= 0)

# [cross-dock] 02: all trucks arrive at the same time at cross-dock
@constraint(cd_modl, cd_truck_arr[k = 1: m, k_prime = 1:m; k != k_prime],
    AT_k[k] - AT_k[k_prime] == 0)

# [cross-dock] 03: total time at the pickup process
@constraint(cd_modl, time_pick[k = 1: m],
    t_pick[o_pick] - AT_k[k] >= 0)

# [cross-dock] 04: total transportation time and processing time does not exceed the planning horizon T_max
@constraint(cd_modl, time_hor[k = 1: m],
    t_pick[o_pick]+ T_o + sum(t_ij[i,j] * x_ijk[i,j,k] for i = vcat(O_del, D) for j = vcat(D, O_dum_del)) - T_max <= 0)


4-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 time_hor[1] : 70 x_ijk[5,6,1] + 70 x_ijk[7,6,1] + 80 x_ijk[5,7,1] + 70 x_ijk[6,7,1] + 60 x_ijk[6,8,1] + 60 x_ijk[7,8,1] + t_pick[1] <= 945.0
 time_hor[2] : 70 x_ijk[5,6,2] + 70 x_ijk[7,6,2] + 80 x_ijk[5,7,2] + 70 x_ijk[6,7,2] + 60 x_ijk[6,8,2] + 60 x_ijk[7,8,2] + t_pick[1] <= 945.0
 time_hor[3] : 70 x_ijk[5,6,3] + 70 x_ijk[7,6,3] + 80 x_ijk[5,7,3] + 70 x_ijk[6,7,3] + 60 x_ijk[6,8,3] + 60 x_ijk[7,8,3] + t_pick[1] <= 945.0
 time_hor[4] : 70 x_ijk[5,6,4] + 70 x_ijk[7,6,4] + 80 x_ijk[5,7,4] + 70 x_ijk[6,7,4] + 60 x_ijk[6,8,4] + 60 x_ijk[7,8,4] + t_pick[1] <= 945.0

In [85]:
#*****************************************
# Objective
#*****************************************

@objective(cd_modl, Min, sum(t_ij[i,j]*x_ijk[i,j,k] for i=vcat(O_dum_pick, P) for j=vcat(P, O_pick) for k=1:m) + sum(t_ij[i,j]*x_ijk[i,j,k] for i=vcat(O_del, D) for j=vcat(D, O_dum_del) for k=1:m))

20 x_ijk[1,2,1] + 20 x_ijk[1,2,2] + 20 x_ijk[1,2,3] + 20 x_ijk[1,2,4] + 30 x_ijk[1,3,1] + 30 x_ijk[1,3,2] + 30 x_ijk[1,3,3] + 30 x_ijk[1,3,4] + 50 x_ijk[2,3,1] + 50 x_ijk[2,3,2] + 50 x_ijk[2,3,3] + 50 x_ijk[2,3,4] + 40 x_ijk[2,4,1] + 40 x_ijk[2,4,2] + 40 x_ijk[2,4,3] + 40 x_ijk[2,4,4] + 50 x_ijk[3,2,1] + 50 x_ijk[3,2,2] + 50 x_ijk[3,2,3] + 50 x_ijk[3,2,4] + 40 x_ijk[3,4,1] + 40 x_ijk[3,4,2] + 40 x_ijk[3,4,3] + 40 x_ijk[3,4,4] + 70 x_ijk[5,6,1] + 70 x_ijk[5,6,2] + 70 x_ijk[5,6,3] + 70 x_ijk[5,6,4] + 80 x_ijk[5,7,1] + 80 x_ijk[5,7,2] + 80 x_ijk[5,7,3] + 80 x_ijk[5,7,4] + 70 x_ijk[6,7,1] + 70 x_ijk[6,7,2] + 70 x_ijk[6,7,3] + 70 x_ijk[6,7,4] + 60 x_ijk[6,8,1] + 60 x_ijk[6,8,2] + 60 x_ijk[6,8,3] + 60 x_ijk[6,8,4] + 70 x_ijk[7,6,1] + 70 x_ijk[7,6,2] + 70 x_ijk[7,6,3] + 70 x_ijk[7,6,4] + 60 x_ijk[7,8,1] + 60 x_ijk[7,8,2] + 60 x_ijk[7,8,3] + 60 x_ijk[7,8,4]

In [ ]:
print(cd_modl)

In [87]:
optimize!(cd_modl)

In [88]:
@show objective_value(cd_modl)

objective_value(cd_modl) = 400.0


400.0

In [ ]:
@show value.(x_ijk);

In [90]:
@show value.(y_i);
prod_qnt = value.(y_i)

value.(y_i) = [0.0, 30.0, 50.0, 0.0, 0.0, 30.0, 50.0, 0.0]


8-element Array{Float64,1}:
  0.0
 30.0
 50.0
  0.0
  0.0
 30.0
 50.0
  0.0

In [91]:
@show value.(AT_k);

value.(AT_k) = [70.0, 70.0, 70.0, 70.0]


In [92]:
@show value.(t_pick);

value.(t_pick) = [70.0]


In [99]:
# vehicle routing information

for k=1:m
    print("\nveh: ", k, "\t")
    for i= 1:n
        for j=1:n
            if value.(x_ijk[i,j,k]) == 1
                print(i, "-->", j, "\t")
            end
        end
    end
end


veh: 1	5-->7	7-->8	
veh: 2	1-->3	3-->4	5-->6	6-->8	
veh: 3	
veh: 4	1-->2	2-->4	

In [109]:
# vehicle estimated arrival time at cross dock

for k=1:m
    print("\nveh: ", k, "\t")
    temp = 0
    for i=1:n
        for j=1:n
            if value.(x_ijk[i,j,k]) == 1
                temp = temp + t_ij[i,j]
                if j == 4 || j==8
                    print("est. arr. time @ CD: ", temp)
                end
            end
        end
    end
end


veh: 1	est. arr. time @ CD: 140
veh: 2	est. arr. time @ CD: 70est. arr. time @ CD: 200
veh: 3	
veh: 4	est. arr. time @ CD: 60

In [152]:
using DataFrames
pick_amt = DataFrame(node =[], amount =[])

for k in keys(y_i)
    val = value.(y_i[k...])
    print(k..., "\t")
    print(val, "\n")
    push!(pick_amt, vcat(k..., val))
end

# pick_amt


1	0.0
2	30.0
3	50.0
4	0.0
5	0.0
6	30.0
7	50.0
8	0.0


In [176]:
movement = DataFrame(x1=[], x2=[], x3=[], x4=[])
for k in keys(x_ijk)
    #print(k, "\t")
    print(getindex.(Tuple.(k), [1 2 3]))
end

LoadError: iteration is deliberately unsupported for CartesianIndex. Use `I` rather than `I...`, or use `Tuple(I)...`